# Импорты

In [ ]:
import itertools
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
pd.options.plotting.backend = "matplotlib"

from ydata_profiling import ProfileReport

In [ ]:
from scipy import stats
import numpy as np

In [ ]:
PLOTS = './plots/'

# Вспомогательные функции

In [ ]:
def hist(df, columns, nrows, ncols, figsize=(600, 1500)):
    fig = make_subplots(rows=nrows, cols=ncols)
    for idx, column in enumerate(columns):
        fig.append_trace(go.Histogram(x=df[column], texttemplate="%{x}", name=column), 
                        idx//ncols + 1, idx%ncols + 1)
    fig.update_layout(
        autosize=False,
        height=figsize[0],
        width=figsize[1],
        margin=dict(
            l=50,
            r=50,
            b=100,
            t=100,
            pad=4
        ),
        # paper_bgcolor="LightSteelBlue",
    )
    fig.show()

In [ ]:
def ind_hists(df, columns, figsize=(400,1500), marginal=None, nbins=None, log=(True, True)):
    for idx, column in enumerate(columns):
        px.histogram(df, column, height=figsize[0], width=figsize[1], marginal=marginal, nbins=nbins, log_x=log[0], log_y=log[1]).show()

In [ ]:
def del_right_outliers(df, column, q):
    q = df[column].quantile(q)
    print(f"Удаление значений, больше {q}. Это {df[df[column] >= q].shape[0]} записей")
    return df[df[column] < q]

In [ ]:
def split_by_zeros(df, columns, threshold):
    idx = (df[columns.keys()] == 0).sum(axis=1) >= threshold
    return df.loc[idx, :], df.loc[~idx, :]

In [ ]:
def value_counts(df, columns):
    for col in columns:
        print(df[col].value_counts(), "\n", "+++", "="*len(col), "+++")

In [ ]:
def create_melt(df, col_substr=None, col_name='Value', appendix_cols: list=None, 
                plot=True, figsize=(1000, 3000), log=(True,False), save_path=None):
    temperature_cols = [col for col in df if col_substr in col] if col_substr else df.columns[4:]
    if appendix_cols:
        temperature_cols += appendix_cols
    print(temperature_cols)
    df_temp = df[temperature_cols]
   
    if plot:
        plot = px.line(df_temp, x='YY-MM-DD HH:00', y=temperature_cols,
                       height=figsize[0], width=figsize[1], log_x=log[0], log_y=log[1], zeroline=True)
        if save_path:
            plot.write_html(save_path)
        plot.show()
    return df_temp

# EDA

## Первичный анализ

- Парсинг временны'х признаков
- Добавление смещения значений по добыче воды за 2 часа
- Фильтрация времени, оставляя ручные измерения (8 утра/вечера)

In [ ]:
df_original.head(3)

- shape=(4334, 59)
- Columns=['Год', 'Месяц', 'День', 'Час', 'Блок манифольда|P бм  |кгс/см²',
       'Блок манифольда|t жидкости| °С', 'С-1|P сеп. |кгс/см²',
       'С-1|t жидкости |°С', 'С-1|L жидкости| см', 'С-2/1|P сеп. |кгс/см²',
       'С-2/1|t жидкости|°С', 'С-2/1|L межфазный|см', 'С-2/1|L нефти|см',
       'С-2/2|P сеп. |кгс/см²', 'С-2/2|t жидкости|°С', 'С-2/2|L межфазный|см',
       'С-2/2|L нефти|см', 'ОН-1/1|P отс. |кгс/см²', 'ОН-1/1|t жидкости|°С',
       'ОН-1/1|L межфазный|см', 'ОН-1/2|P отс. |кгс/см²',
       'ОН-1/2|t жидкости|°С', 'ОН-1/2|L  межфазный|см', 'С-3|t жидкости|°С',
       'С-3|L нефти|см', 'П-1|Т нефти на входе|°С', 'П-1|Т нефти на выходе|°С',
       'П-1|Р нефти на входе|кгс/см²', 'П-1|Р нефти на выходе|кгс/см²',
       'П-1|Т теплоно-сителя|°С', 'П-1|Т дымовых газов|°С',
       'П-1|Р на горелку|кгс/см2', 'П-2|Т нефти на входе|°С',
       'П-2|Т нефти на выходе|°С', 'П-2|Р нефти на входе|кгс/см²',
       'П-2|Р нефти на выходе|кгс/см²', 'П-2|Т теплоно-сителя|°С',
       'П-2|Т дымовых газов|°С', 'П-2|Р на горелку|кгс/см3',
       'П-3|Т нефти на входе|°С', 'П-3|Т нефти на выходе|°С',
       'П-3|Р нефти на входе|кгс/см²', 'П-3|Р нефти на выходе|кгс/см²',
       'П-3|Т теплоно-сителя|°С', 'П-3|Т дымовых газов|°С',
       'П-3|Р на горелку|кгс/см4', 'БЕВ-1|L воды|см', 'БЕВ-1|V  воды|м3',
       'БЕВ-1|V нефти|м3', 'БЕВ-1|t воды|°С', 'БЕВ-2|L воды|см',
       'БЕВ-2|V  воды|м3', 'БЕВ-2|V нефти|м3', 'БЕВ-2|t воды|°С',
       'БЕВ-3|L воды|см', 'БЕВ-3|V  воды|м3', 'БЕВ-3|V нефти|м3',
       'БЕВ-3|t воды|°С', 'Вода с лагом (-1)']
- float64(22), int32(4), int64(33)
- Null значений нету

In [ ]:
df_prep = pd.read_csv(f'filled_outliers_lag{lag}.csv')
profile = ProfileReport(df_prep, title="Profiling Report")
profile.to_file(output_file='report.html')

## Анализ нулевых значений

In [ ]:
zero_columns = {column: (df[column]==0).sum()/df[column].count() * 100 
                for column 
                in df.columns
                if (df[column]==0).sum()!=0}

zero_columns = dict(sorted(zero_columns.items(), 
                           key=lambda item: item[1], 
                           reverse=True))
print(*[f'{idx:2}.  {zero_percent:5.2f}: {column}' for idx, (column, zero_percent) in enumerate(zero_columns.items())], 
        sep='\n')

Уберем из списка нулевых столбцов те, у которых меньше 1% нулей

In [ ]:
zero_columns = slice(zero_columns, stop=20)


Alarm:

 0.  100.00: БЕВ-2|V нефти|м3
 1.  100.00: БЕВ-3|V нефти|м3
 2.  95.34: БЕВ-1|V нефти|м3
 3.  80.94: БЕВ-1|t воды|°С
 4.  80.94: БЕВ-2|t воды|°С
 5.  80.94: БЕВ-3|t воды|°С
 6.  44.23: П-1|Т нефти на входе|°С
 7.  44.23: П-1|Т нефти на выходе|°С
 8.  44.23: П-1|Р нефти на входе|кгс/см²
 9.  44.23: П-1|Р нефти на выходе|кгс/см²
10.  44.23: П-1|Т теплоно-сителя|°С
11.  44.23: П-1|Т дымовых газов|°С
12.  44.23: П-1|Р на горелку|кгс/см2

In [ ]:
bevs = list(slice(zero_columns, stop=6).keys())
hist(df, bevs, 3, 2, figsize=(800,1500))

- В БЕВ'ав слишком много нулевых значений => их больше не учитываем

In [ ]:
df.drop(columns=bevs, inplace=True)
zero_columns = slice(zero_columns, start=6)
zero_columns

In [ ]:
p_1 = list(slice(zero_columns, stop=7).keys())
ind_hists(df, p_1, marginal='box', nbins=200, log=(False,True))

> Выбросы в: 
> - "П-1|Р нефти на выходе|кгс/см²" (43.1, count=1)
> - "П-1|Р на горелку|кгс/см2': 44.23165666820489" (300, count=1)

> Удаляем...

In [ ]:
df = del_right_outliers(df, 'П-1|Р нефти на выходе|кгс/см²', .999)
df = del_right_outliers(df, 'П-1|Р на горелку|кгс/см2', .9991)


> Снова смотрим на распределения

In [ ]:
ind_hists(df, p_1, marginal='box', nbins=200, log=(False,True))

> Стало немного лучше

In [ ]:
df.shape, df_original.shape

> Однако, в первой печке тоже много значений по нулям
- Посмотрим, сколько строк имеют большое кол-во нулей

In [ ]:
threshold = 7
df_much_zeros, df_without_zeros = split_by_zeros(df, zero_columns, threshold)
print(f"Датасет содержащий строки с кол-во нулей в столбцах < {threshold}: shape=({df_without_zeros.shape})")
print(f"Датасет содержащий строки с кол-вом нулей в столбцах >= {threshold}: shape=({df_much_zeros.shape})")

In [ ]:

DATA = './data'
df = Dataset(os.path.join(DATA, 'processed.xlsx'))
df.parse_datetime()

df.set_lag(lag=-1)
df.set_lag(lag=1)


cols_for_delete = [
    
       # оставляем пока что П-_|Т нефти на входе|°С
       #  'П-1|Т нефти на входе|°С',
       #  'П-2|Т нефти на входе|°С',
       #  'П-3|Т нефти на входе|°С',
       # median аггрегация => П-123median|Т нефти на выходе|°С
       #  'П-1|Т нефти на выходе|°С',
       #  'П-2|Т нефти на выходе|°С',
       #  'П-3|Т нефти на выходе|°С',
       # max аггрегация => П-123max|Р нефти на входе|кгс/см²
       #  'П-1|Р нефти на входе|кгс/см²',
       #  'П-2|Р нефти на входе|кгс/см²',
       #  'П-3|Р нефти на входе|кгс/см²',
       # max аггрегация => П-123max|Р нефти на выходе|кгс/см²
       #  'П-1|Р нефти на выходе|кгс/см²',
       #  'П-2|Р нефти на выходе|кгс/см²',
       #  'П-3|Р нефти на выходе|кгс/см²',
       # константные значения
       #  'П-1|Р на горелку|кгс/см2',
       #  'П-2|Р на горелку|кгс/см3',
       #  'П-3|Р на горелку|кгс/см4',
       # БЕВ'ы 
       'БЕВ-1|L воды|см', 'БЕВ-1|V  воды|м3',
       'БЕВ-1|V нефти|м3', 'БЕВ-1|t воды|°С', 'БЕВ-2|L воды|см',
       'БЕВ-2|V  воды|м3', 'БЕВ-2|V нефти|м3', 'БЕВ-2|t воды|°С',
       'БЕВ-3|L воды|см', 'БЕВ-3|V  воды|м3', 'БЕВ-3|V нефти|м3',
       'БЕВ-3|t воды|°С',
       ]

print(df.df.columns)

df.df.drop(cols_for_delete, axis=1, inplace=True)

columns = df.df.columns[4:]
quantiles = [[0.001, 0.999]]*len(columns)
threshold = [[3]]*len(columns)
df.recovery_outliers(columns, quantiles=quantiles, threshold=threshold)

## Обработка выбросов

In [ ]:
def preprocess(lag, cols_for_delete, outliers=True, insert=True):
    df_original, df = load_dataset('processed.xlsx', dropna=True, lag=lag)
    df['Год'] = 2021
    print(f"Удаление неинформативных признаков: \n{cols_for_delete}")
    df = df.drop(cols_for_delete,axis=1)

    # Восстановление значений
    if outliers:
        columns = df.columns[4:]
        quantiles = [[0.001, 0.999]]*len(columns)
        df = fill_outliers(df, columns, quantiles=quantiles, insert=insert)

    df.insert(0, 'YY-MM-DD HH:00', df[['Год', 'Месяц','День','Час']].apply(
        lambda x: f'{x[0]:04}-{x[1]:02}-{x[2]:02} {x[3]:02}:00', 
        axis = 1)) 
    df = df.sort_values('YY-MM-DD HH:00')
    df.to_csv(f'filled_outliers_lag{lag}.csv', index=False)
    return df



In [ ]:
lag=-1
df = preprocess(lag, cols_for_delete, False, False)


pechki = [col for col in df if 'Р нефти на выходе' in col]
df['П-123max|Р нефти на выходе|кгс/см²'] = df[pechki].max(axis=1)
pechki = [col for col in df if 'Р нефти на входе' in col]
df['П-123max|Р нефти на входе|кгс/см²'] = df[pechki].max(axis=1)
pechki = [col for col in df if 'Т нефти на выходе' in col]
df['П-123median|Т нефти на выходе|кгс/см²'] = df[pechki].median(axis=1)
pechki = [col for col in df if 'Т теплоно-сителя' in col]
df['П-123median|Т теплоно-сителя|кгс/см²'] = df[pechki].median(axis=1)

columns = df.columns[5:]
quantiles = [[0.001, 0.999]]*len(columns)
df = fill_outliers(df, columns, quantiles=quantiles, insert=True)


In [ ]:
outlier_columns = [col for col in df if 'outlier' in col]
outlier_columns.append('Вода с лагом (-1)')


orig_columns = [col for col in df if col not in outlier_columns]
orig_columns.append('Вода с лагом (-1)')
orig_columns.remove('YY-MM-DD HH:00')

pechki_columns = [col for col in df if 'П' in col]
pechki_columns.append('Вода с лагом (-1)')


### Scatter matrixes

In [ ]:
for i in range(0, len(outlier_columns), 10):
    fig = px.scatter_matrix(df[outlier_columns[i: i+11]], height=4000, width=4000)
    fig.update_traces(diagonal_visible=False, showlowerhalf=False)
    fig.write_html(f'ебейшая_scatter_matrix{i}.html')

In [ ]:
px.scatter_matrix(df[orig_columns])

In [ ]:
px.scatter_matrix(df[pechki_columns])

### Correlation matrixes

In [ ]:
def show_corr_matrix(df, columns, method='pearson', filename='Default'):
    sns_colormap = [[0.0, '#3f7f93'],
                    [0.1, '#6397a7'],
                    [0.2, '#88b1bd'],
                    [0.3, '#acc9d2'],
                    [0.4, '#d1e2e7'],
                    [0.5, '#f2f2f2'],
                    [0.6, '#f6cdd0'],
                    [0.7, '#efa8ad'],
                    [0.8, '#e8848b'],
                    [0.9, '#e15e68'],
                    [1.0, '#da3b46']]

    corr = df[columns].corr(method)
    fig = px.imshow(corr, text_auto=True, height=2000, width=2000,  color_continuous_scale=sns_colormap)
    fig.update_traces(textfont_size=10,  texttemplate = "%{z:.2f}")
    fig.write_html(f'{filename}.html')

In [ ]:
show_corr_matrix(df, outlier_columns, filename='corr_matrix_with_filled_outliers')
show_corr_matrix(df, orig_columns, filename='corr_matrix_with_orig_columns')


In [ ]:
corr = df[orig_columns].corr()
px.imshow(corr, text_auto=True, height=1500)

In [ ]:
corr = df[pechki_columns].corr()
px.imshow(corr, text_auto=True, height=1500)

##  Проверка колебаний аналогичных показателей во времени

In [ ]:
pechki = [col for col in df if 'П' in col]
pechki.append('Добыча воды за 2 ч |м3')

# pechki = ['Добыча воды за 2 ч |м3']
pechki

In [ ]:
np.polyfit(df['Добыча воды за 2 ч |м3'], df.index, 3)

In [ ]:
px.line(df, x='YY-MM-DD HH:00', y=pechki, height=1000, width=2000)


### Температуры

In [ ]:
create_melt(df, 
            
            plot=True,
            log=(False,False),
            save_path=f'all_lag{lag}.html'
            )[:25]

In [ ]:
create_melt(df, '°С', 'Температура', 
            appendix_cols=[f'Вода с лагом ({lag})'],
            plot=True,
            log=(False,True),
            save_path=f'temperature_time_series_without_outliers_lag{lag}.html'
            )[:25]

### Давления

In [ ]:
create_melt(df, 'кгс/см', 'Давление',
            plot=True,
            log=(False,True),
            save_path=f'pressure_time_series_without_outliers_lag{lag}.html')

- "П-3|Р на горелку|кгс/см4", "П-2|Р на горелку|кгс/см3", "П-1|Р на горелку|кгс/см2" пересекаются

### Уровни

In [ ]:
create_melt(df, 'см', 'Уровень', 
            log=(False,True),
            save_path=f'level_time_series_without_outliers_lag{lag}.html')